OpenStreetMap Project

Map Area:
Sydney, NSW, Australia

https://www.openstreetmap.org/relation/5750005

https://mapzen.com/data/metro-extracts/metro/sydney_australia/

The map is the city where I am living. I will explore the map data to find interesting things.

Data Overview:

Sydney_Australia.osm ......... 303.9 MB

Sydney_Australia.db .......... 162.1 MB

nodes.csv ............. 112.3 MB

nodes_tags.csv ........ 5.3 MB

ways.csv .............. 11 MB

ways_tags.csv ......... 38.7 MB

ways_nodes.cv ......... 21.2 MB

Data Wrangling:
    
In this project, I apply SQL database (sqlite) to store the map data. Before importing all data into the database, I process data wrangling to fix errors in the dataset:

1. I apply regular expressions to check consistency of the data in the nodes.csv and ways.csv files. The nodes.csv and ways.csv files store the basic information of nodes and ways in the map, e.g., users who mark the nodes and ways, coordinates and identities of nodes and ways.
2. I check the nodes_tags and ways_tags files, which store the descriptions of nodes and ways. I find some errors in the file and fix these errors when generating all these csv files.
3. I explore the ways_nodes.csv file, which describes the relations between nodes and ways. As all the data in ways_nodes.csv are generated from nodes.csv and ways.csv files, it is not necessary to clean ways_nodes data separately.

Problems in OSM DATA:

I find some problems in the dataset as follows:

1. In nodes_tags.csv and ways_tags.csv, I find two a string key named "city_1" with values "Sydney" and "Roseberry". As I notice that all the other keys describing city are named 'city', I think 'city_1' should be a manual input error. As these are only two errors in this case, I fix the errors manually.

2. I find that some of the suburb names under tag k = 'addr:city' are not the names of Sydney suburbs. That may because some of these suburb may be famous places rather than a offical suburb, e.g., Darling Harbour, or they belong to other regions near Sydney, e.g., central coast. Thus, I keep these suburb names. On the other name, some names of Sydney suburbs are not formatted correctly. For example, wentworth Point, the first 'w' is not capitcal. Thus, I write a piece of codes to make sure all the names starting with capital letters. The codes are shown in the following paragraph.

3. A valid Sydney postcode is a 4-digit number starting with '2'. I check all the postcodes and find some invalid postcodes as follows:  
['NSW 2010',
 'NSW 2000',
 '210',
 'NSW 2010',
 'NSW 2010',
 'NSW 2567',
 'NSW 2068',
 'NSW 2037',
 'NSW 2000',
 '1568',
 'NSW 2120',
 'NSW 2120',
 'NSW 2120',
 'NSW 2010',
 'NSW 2000',
 'NSW 2077',
 'NSW 2000',
 'NSW 2000',
 'NSW 2022',
 'NSW 2010',
 '1640',
 'NSW 2021',
 'NSW 2000',
 'NSW 2011']  
Obviously, 'NSW' can be omitted since Sydney is just in NSW. In addition, '210', '1568' and '1640' should be error postcodes. After manually checking, I find '1568' and '1640' are  valid postcodes (I have no idea why), but '210' should be '2010'. Thus, I fix this manually.  
After that, I write a piece of codes to delete 'NSW' before postcodes. The codes are shown in the following paragraph.

4. I check all the address names and find some street names are incomplete or abbreviate. I find the following problematic street names:  
[Addison road
Shaw
Shaw
Shaw
Berith
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Berith
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Berith
Wolli
Wolli
Wolli
Berith
Berith
Edward
Berith
Berith
Edward
Berith
Edward
Berith
Edward
Berith
Edward
Berith
Berith
Wolli
Berith
Wolli
Berith
Wolli
Berith
Wolli
Berith
Wolli
Berith
Berith
Wolli
Berith
Wolli
Berith
Wolli
Berith
Wolli
Berith
Wolli
Berith
Wolli
Wolli
Wolli
Wolli
Wolli
Berith
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Berith
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Wolli
Chalmers street
York St
Liverpool St
Argyle St
george street
George street
George st
Roberts Ave
Henderson St
Margaret St
Plowman street
Pittwater Rd
5 Active Pl].  
Here, 'Wolli', 'Berith', 'Edward' and 'Shaw' are all provided by the same user '2046smh', who omitted all 'Street' after these names. Thus, I write a piece of codes to add 'Street' after names.  
In addition, I fix all the abbreviate names using the codes shown in the following paragraph.

In [1]:
# fix suburb names
'''
def fix_suburb_names(name_string):
    name_list = name_string.split(' ')
    name_tmp = []
    for name in name_list:
        name_tmp.append( name[0].upper() + name[1:].lower() )
    return ' '.join(name_tmp)

if tag_dict['type'] == 'addr' and tag_dict['key'] == 'city':
    tag_dict['value'] = fix_suburb_names( tag_dict['value'] )
'''

# fix postcodes
'''
def fix_postcodes(postcode):
    nsw_re = re.compile( r'NSW' )
    if nsw_re.search( str(postcode) ): 
        return postcode.split(' ')[1]
    else:
        return postcode

if tag_dict['type'] == 'addr' and tag_dict['key'] == 'postcode':
    tag_dict['value'] = fix_postcodes( tag_dict['value'] )
    
'''

# fix street names
'''
name_mapping = { "St": "Street", "St.": "Street", "st": "Street", "street": "Street", "Ave": "Avenue", "Av": "Avenue", "Rd.": "Road", "Rd": "Road", "road": "Road", "Pl": "Place"}

def fix_street_name(name_string):
    street_type = name_string.split(' ')[-1]
    if street_type in name_mapping.keys():
        return name_string[:-len(street_type)] + name_mapping[street_type]
    else:
        return name_str
        
if tag_dict['type'] == 'addr' and tag_dict['key'] == 'street':
    tag_dict['value'] = fix_street_name( tag_dict['value'] )
if tag_dict['type'] == 'addr' and tag_dict['key'] == 'street' and tag_dict['value'] in ['Shaw', 'Wolli', 'Edward', 'Berith']:
    tag_dict['value'] = tag_dict['value'] + ' Street'
'''

"\ndef fix_postcodes(postcode):\n    nsw_re = re.compile( r'NSW' )\n    if nsw_re.search( str(postcode) ): \n        return postcode.split(' ')[1]\n    else:\n        return postcode\n\nif tag_dict['type'] == 'addr' and tag_dict['key'] == 'postcode':\n    tag_dict['value'] = fix_postcodes( tag_dict['value'] )\n"

Data Exploration:
    
Number of nodes:  
SELECT COUNT(*) FROM nodes;  
1352655

Number of ways:  
SELECT COUNT(*) FROM ways;   
185519

Number of unique users:       
SELECT COUNT(DISTINCT(a.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) AS a;   
1810

Top 10 contributing users:  
SELECT a.user, COUNT(a.user) AS num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) AS a GROUP BY a.user ORDER BY num DESC LIMIT 10;

"behemoth14"   "117535"  
"inas"	       "92373"  
"TheSwavu"	   "69629"  
"ChopStiR"	   "58134"  
"aharvey"	   "47864"  
"Leon K"	   "47615"  
"cleary"	   "45911"  
"Rhubarb"	   "40812"  
"Warin61"	   "40583"  
"AntBurnett"   "38062"  

Top 10 types of shops in Sydney:  
CREATE VIEW nodes_ways_tags AS SELECT * FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags);  
SELECT value, COUNT() as num FROM nodes_ways_tags WHERE key = 'shop' GROUP BY value ORDER BY num DESC LIMIT 10;  

"supermarket"	    "447"  
"convenience"	    "301"  
"mall"	            "167"  
"alcohol"	        "126"  
"bakery"	        "99"  
"car_repair"	    "99"  
"hairdresser"	    "97"  
"clothes"	        "93"  
"bicycle"	        "76"  
"department_store"	"76"  

Top 10 types of amenities in Sydney:  
SELECT value, COUNT() as num FROM nodes_ways_tags WHERE key = 'amenity' GROUP BY value ORDER BY num DESC LIMIT 10;  

"parking"	"3826"  
"bench"	"1257"  
"school"	"1014"  
"restaurant"	"856"  
"cafe"	"774"  
"toilets"	"717"  
"place_of_worship"	"588"  
"drinking_water"	"583"  
"fast_food"	"577"  
"bicycle_parking"	"482"  

Top 10 nodes including the most tags and their users:  
SELECT nodes_tags.id, nodes.user, COUNT(nodes_tags.id)  as num FROM nodes JOIN nodes_tags ON nodes.id = nodes_tags.id GROUP BY nodes_tags.id ORDER BY num DESC LIMIT 10;  

"1191079691"	"malcolmh"	"33"  
"13766899"	"Jojo4u"	"31"  
"288139026"	"Bryce C Nesbitt"	"26"  
"250291346"	"Bryce C Nesbitt"	"25"  
"1618699976"	"mrpulley"	"25"  
"20930157"	"ntppr"	"19"  
"4127353279"	"Rhubarb"	"19"  
"2524742470"	"morray"	"18"  
"3023984134"	"Alex Brak"	"18"  
"4115333570"	"Rhubarb"	"18"  

Top 10 ways including the most tags and their users:  
SELECT ways_tags.id, ways.user, COUNT(ways_tags.id)  as num FROM ways JOIN ways_tags ON ways.id = ways_tags.id GROUP BY ways_tags.id ORDER BY num DESC LIMIT 10;  

"4960757"	"KatsuOhata"	"33"  
"5011620"	"Leon K"	"23"  
"248793312"	"samuelrussell"	"23"  
"409047656"	"AlexOnTheBus"	"23"  
"304056519"	"aharvey"	"22"  
"341469879"	"samuelrussell"	"22"  
"407606350"	"Biff"	"22"  
"409047647"	"AlexOnTheBus"	"22"  
"5011712"	"Leon K"	"21"  
"10208997"	"Biff"	"21"  

The number of fire station in Sydney:  
SELECT COUNT(value) AS 'Num of Fire Station' FROM nodes_ways_tags WHERE value = 'fire_station';  
103

Types of religions and numbers of places of worship:  
SELECT nodes_tags.value as 'Place of Worship', COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT id FROM nodes_tags WHERE value='place_of_worship')  as a
    ON nodes_tags.id=a.id
WHERE nodes_tags.key='religion'
GROUP BY nodes_tags.value
ORDER BY num DESC;

"christian"	"323"  
"muslim"	"8"  
"buddhist"	"4"  
"caodaism"	"1"  
"hindu"	"1"  
"sikh"	"1"  

Top 10 of denominations:  
SELECT a.value, COUNT(a.value) AS num FROM nodes_tags AS a WHERE a.key = 'denomination' GROUP BY a.value ORDER BY num DESC LIMIT 10;  

"anglican"	"57"
"catholic"	"57"
"uniting"	"44"
"baptist"	"22"
"presbyterian"	"21"
"roman_catholic"	"11"
"seventh_day_adventist"	"6"
"congregational"	"3"
"greek_orthodox"	"3"
"mormon"	"3"

Top 10 ways including the most nodes:  
SELECT id, COUNT(id) FROM ways_nodes GROUP BY id ORDER BY COUNT(id) DESC LIMIT 10;  

"149668786"	"1789"  
"150905541"	"1180"  
"310457827"	"1125"  
"149949600"	"1110"  
"149553706"	"943"  
"398644134"	"935"  
"414632940"	"923"  
"224602856"	"901"  
"366387557"	"841"  
"225080521"	"837"

Top 10 operators in Sydney:  
SELECT a.value, COUNT(a.value) AS num FROM nodes_ways_tags AS a WHERE a.key = 'operator' GROUP BY a.value ORDER BY num DESC LIMIT 10;  

"RailCorp"	"2096"  
"Sydney Trains"	"238"  
"Australia Post"	"137"  
"The University of Sydney"	"118"  
"Telstra"	"90"  
"Sydney Water"	"74"  
"The University of Western Sydney"	"51"  
"Energy Australia"	"43"  
"Sydney Catchment Authority"	"37"  
"City of Sydney"	"32"  

Improvement Suggestion:

When I clean these data, a poblem in the dataset bothers me much, which is a key may have multiple types of values and express different meanings. For example, when checking the types of buildings, I find the following results: 

SELECT a.value, COUNT(a.value) AS num FROM nodes_ways_tags AS a WHERE a.key = 'building' GROUP BY a.value ORDER BY num DESC LIMIT 10;  

"yes"	"16377"  
"house"	"9567"  
"garage"	"3690"  
"apartments"	"1387"  
"residential"	"706"  
"industrial"	"565"  
"school"	"552"  
"commercial"	"546"  
"university"	"486"  
"retail"	"387"  

Obviously 'yes' is not a building type, here which may represent that there exists a building. Thus, key "building" includes two types of values, and thus data type consistency cannot be maintained, which may confuse users of such a dataset. And this problem cannot be casued by manual input errors due to the large number of this situations. The problem is probably caused by the problematic data format standard.

Hence, I suggest to improve the data format standard to make sure one key can only have one type of values, so that a user can identify the exact information he/she wants without such noises.

For the key 'building', I can easily write codes to set all {'building': 'yes'} to {'is_building': 'yes'}. However, it is quite hard to solve all similar data problems since such a problem can only be identified through manually checking. Hence, an improved data format standard should be expected and all contributors of OSM are encouraged to strictly follow the standard. 